In [1]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)

examples = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    }
]


prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give short answers."),
        example_prompt,
        ("human", "Could you describe the movie {movie} with 3 emoticons??"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("The Ironman")

content='🔥🦸\u200d♂️💥'


In [3]:
invoke_chain("The Titanic")

content='🚢💑❄️'


In [4]:
invoke_chain("Home Alone")

content='🏠😱🎄'


In [5]:
load_memory(_)

[SystemMessage(content='The human mentions Ironman, and the AI responds with emojis representing fire, a superhero, and explosion.'),
 HumanMessage(content='The Titanic'),
 AIMessage(content='🚢💑❄️'),
 HumanMessage(content='Home Alone'),
 AIMessage(content='🏠😱🎄')]